In [4]:
'''
    local_exec
    IBM/Dell Qiskit Runtime Emulator Demo
    July 26, 2021
    
    
    The following notebook walks through a (simple) example usage of the 
    Qiskit Runtime Emulator in a local execution environment: i.e. potentially
    using a locally installed simulator or a remote call directly from a
    local machine to a remote simulator or QPU.
'''

from qiskit_emulator import DellHybridProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os

'''
    If the program that interacts with the simulator/QPU is small enough,
    it can be stored as a string in the file that interfaces with the 
    provider. Both directories and files can be taken as input, as well.

'''

RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results using either simulator or QPU backend.
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

In [5]:
'''
    The DellHybridProvider is an interface that offers a choice of runtime; either local or remote
'''

provider = DellHybridProvider()

In [6]:
'''
    The runtime is a service provided that allows clients to upload, update,
    view, and run programs inside an execution environment. Since the client
    has not specified a remote runtime to the provider it defaults to local.
'''

program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {program_id}")

DEBUG:qiskit_emulator.emulator_runtime_service:Have string: 
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results using either simulator or QPU backend.
    result = backend.run(circuits, **kwargs).result()

    user_mess

PROGRAM ID: 682f340e7a7fef02


In [7]:
programs = provider.runtime.pprint_programs(refresh=True)

682f340e7a7fef02:
  Name: 682f340e7a7fef02
  Description: Qiskit test program
  Version: 0
  Creation date: 12/08/2021 17:36:58
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [8]:
provider.runtime.update_program(program_id, description="IBM/Dell Updated Qiskit Runtime Program")

True

In [9]:
programs = provider.runtime.pprint_programs(refresh=True)

682f340e7a7fef02:
  Name: 682f340e7a7fef02
  Description: IBM/Dell Updated Qiskit Runtime Program
  Version: 0
  Creation date: 12/08/2021 17:36:58
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [10]:
'''
    Setting up QuantumCircuit objects from Qiskit to provide as inputs
    to the program we uploaded.
'''

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [11]:
'''
    Through the provider we can run an instance of our program with the
    inputs we have just created. 
    
    When we run a job locally, we launch a separate process and communicate with the
    client process via a socket connection.
'''

job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

DEBUG:qiskit_emulator.emulator_runtime_job:50395
DEBUG:root:starting to listen to port 50395
DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmpq7qsj28n
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpq7qsj28n/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpq7qsj28n/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpq7qsj28n/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpq7qsj28n/executor.py']
DEBUG:root:accepted client connection from ('127.0.0.1', 37780)
DEBUG:qiskit_emulator.emulator_runtime_job:result: got final result.
DEBUG:qiskit_emulator.emulator_runtime_job:local thread: exiting
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpq7qsj28n/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: using default backend: aer

In [12]:
'''
    We can obtain job results and specify a timeout if desired.
    If we do not specify a timeout, the function call returns immediately,
    either with None or the final results if they are present.
'''

results = job.result(timeout=60)
print(results)

{'backend_name': 'aer_simulator', 'backend_version': '0.8.2', 'qobj_id': 'e28c6c57-2574-438d-b46a-02c37d920ff1', 'job_id': 'f86ca89a-92a0-4db3-9d21-8f12b6ac90ae', 'success': True, 'results': [{'shots': 1024, 'success': True, 'data': {'counts': {'0x0': 487, '0x3': 537}}, 'meas_level': 2, 'header': {'clbit_labels': [['c', 0], ['c', 1]], 'creg_sizes': [['c', 2]], 'global_phase': 0.0, 'memory_slots': 2, 'metadata': None, 'n_qubits': 2, 'name': 'circuit-2', 'qreg_sizes': [['q', 2]], 'qubit_labels': [['q', 0], ['q', 1]]}, 'status': 'DONE', 'seed_simulator': 2258615556, 'metadata': {'parallel_state_update': 2, 'measure_sampling': True, 'method': 'stabilizer', 'parallel_shots': 1, 'device': 'CPU', 'fusion': {'enabled': False}}, 'time_taken': 0.002226786}], 'date': '2021-08-12T17:37:49.800321', 'status': 'COMPLETED', 'header': {'backend_name': 'aer_simulator', 'backend_version': '0.8.2'}, 'metadata': {'time_taken': 0.002396849, 'parallel_experiments': 1, 'omp_enabled': True, 'max_gpu_memory_mb'

In [13]:
'''
    We can also provide a callback function to the runtime.
    A thread launched in the client process that polls for messages
    calls the callback function whenever it receives a non-final message.
'''

def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')
    

In [14]:
job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

DEBUG:qiskit_emulator.emulator_runtime_job:34955
DEBUG:root:starting to listen to port 34955
DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmpxzsk1_f3
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpxzsk1_f3/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpxzsk1_f3/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpxzsk1_f3/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpxzsk1_f3/executor.py']
DEBUG:root:accepted client connection from ('127.0.0.1', 32854)
DEBUG:qiskit_emulator.emulator_runtime_job:result: got final result.
DEBUG:qiskit_emulator.emulator_runtime_job:local thread: exiting
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpxzsk1_f3/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: using default backend: aer

******************


From Callback Function: {'results': 'intermittently'}


******************


In [15]:
'''
    We can also specify a different backend to run the quantum portions on. 
    The default backend is the Aer simulator.
'''

program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

DEBUG:qiskit_emulator.emulator_runtime_job:56871
DEBUG:root:starting to listen to port 56871
DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmpd783vmz_
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpd783vmz_/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpd783vmz_/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpd783vmz_/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpd783vmz_/executor.py']
DEBUG:root:accepted client connection from ('127.0.0.1', 35720)
DEBUG:qiskit_emulator.emulator_runtime_job:result: got final result.
DEBUG:qiskit_emulator.emulator_runtime_job:local thread: exiting
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpd783vmz_/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: {'circuits': <qiskit.circu

******************


From Callback Function: {'results': 'intermittently'}


******************
